In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from vllm import LLM
from vllm import LLM, SamplingParams
from datasets import load_dataset

/home/lab/.conda/envs/sdg_research_upstream/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 03-20 18:20:58 [__init__.py:256] Automatically detected platform cuda.


2025-03-20 18:20:59,120	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
#tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama-3_3-Nemotron-Super-49B-v1") # microsoft/phi-4
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-4")

In [4]:
thinking = "on"
print(tokenizer.apply_chat_template([{"role": "system", "content": f"detailed thinking {thinking}"}, {'role': 'user', 'content': 'Hello, how are you?'}], add_generation_prompt=True, tokenize=False))

<|im_start|>system<|im_sep|>detailed thinking on<|im_end|><|im_start|>user<|im_sep|>Hello, how are you?<|im_end|><|im_start|>assistant<|im_sep|>


In [ ]:
llm = LLM(
            model= "nvidia/Llama-3_3-Nemotron-Super-49B-v1", #"/new_data/experiments_rh/ab-bmo-phi-4-mini-instruct/lr_5e-06_bs_128_bmo_lab_stack_w_doc_outline/hf_format/samples_32218",
            gpu_memory_utilization=0.8,
            enable_prefix_caching=False,
            seed=42,
            tensor_parallel_size=8,
            trust_remote_code=True
        )


In [5]:
tokenizer.chat_template

"{% for message in messages %}{% if (message['role'] == 'system') %}{{'<|im_start|>system<|im_sep|>' + message['content'] + '<|im_end|>'}}{% elif (message['role'] == 'user') %}{{'<|im_start|>user<|im_sep|>' + message['content'] + '<|im_end|>'}}{% elif (message['role'] == 'assistant') %}{{'<|im_start|>assistant<|im_sep|>' + message['content'] + '<|im_end|>'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant<|im_sep|>' }}{% endif %}"

### Start Teacher Model
```shell
export HUGGINGFACE_HUB_CACHE="/new_data/hf_cache"   
export HF_DATASETS_CACHE="/dev/shm/hf"
export HF_HOME="/new_data/hf_cache"
export HF_MODEL_CACHE="/new_data/hf_cache"

port=8000
for i in {0..7}; do
    CUDA_VISIBLE_DEVICES=$i python -m vllm.entrypoints.openai.api_server \
        --model microsoft/phi-4 \
        --dtype float16 \
        --tensor-parallel-size 1 \
        --port $port \
        --trust-remote-code > run_phi4_teacher_model_$i.log 2>&1 &
    port=$((port + 1))
done
```

### Run SDG in parallel
```shell
# Get dataset size and save into variable
dataset_size=$(wc -l /new_data/knowledge_rh/quality/base_datasets/quality_seed_example_chunked.jsonl | awk '{print $1}')
number_of_processes=8
port=8000
for i in {0..7}; do
    dataset_start_index=$((i * dataset_size / number_of_processes))
    dataset_end_index=$((dataset_start_index + dataset_size / number_of_processes))
    python scripts/generate.py --ds_path  /new_data/knowledge_rh/quality/base_datasets/quality_seed_example_chunked.jsonl \
        --bs 2 --num_workers 30 \
        --save_path /new_data/knowledge_rh/quality/knowledge1.25_extractive_phi4/gen.jsonl \
        --flow /home/lab/abhi/SDG-Research-Upstream/src/instructlab/sdg/flows/generation/knowledge/synth_knowledge1.5_phi4.yaml \
        --endpoint http://localhost:$port/v1 \
        --checkpoint_dir /new_data/knowledge_rh/quality/knowledge1.25_extractive_phi4/data_checkpoints \
        --save_freq 1000 \
        --dataset_start_index $dataset_start_index \
        --dataset_end_index $dataset_end_index > run_sdg_$i.log 2>&1 &
    echo "Starting process $i with dataset from $dataset_start_index to $dataset_end_index on port $port"
    port=$((port + 1))
done
```